#### Question 1
Given the following specification for a single-formant resonator, obtain the transfer function of the filter H(z) from the relation between resonance frequency / bandwidth, and the pole angle / radius. Plot filter magnitude response (dB magnitude versus frequency) and impulse response.  

* F1 (formant) = 900 Hz
* B1(bandwidth) = 200 Hz
* Fs (sampling freq) = 16 kHz

In [1]:
import numpy as np
from scipy.signal import zpk2tf,freqz,sawtooth,square,impulse
from math import pi
from numpy import exp,zeros_like,cos,sin,log10,angle
from numpy import convolve as conv

import matplotlib
matplotlib.use('PS')
import pylab as plt
plt.switch_backend('PS')

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode']=True
plt.style.use(['bmh'])
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
#plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

plt.rcParams.update({"axes.facecolor" : "white",
                     "axes.edgecolor":  "black"})

<ipython-input-1-2707b7cb2082>:13: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  plt.rcParams['text.latex.unicode']=True


In [2]:
f1 = 900 #formant frequency
b1 = 200 #bandwidth
fs = 16000 #sampling frequency
ts = 1.0/fs # sampling time

In [3]:
r = np.exp(-pi*b1*ts)
theta = 2*pi*f1*ts

poles = [r*exp(1j*theta) , r*exp(-1j*theta)]
zeros = zeros_like(poles)

In [4]:
b,a = zpk2tf(zeros,poles,k=1)
print(b,a)
print(-2*r*cos(theta),r**2)

[1. 0. 0.] [ 1.         -1.80412535  0.92446525]
-1.8041253513834825 0.9244652503762558


In [5]:
w,h = freqz(b,a)
plt.figure()
plt.subplot(1,2,1)
plt.plot((fs * w/(2*pi))/1000.0,20*log10(abs(h)),'b')
s="Frequency Response of Vocal Tract with F1: {} and B1: {}"
plt.suptitle(s.format(f1,b1),fontsize=12)
plt.title(r"Magnitude response",fontsize=12)
plt.ylabel(r"$|H(\Omega|$ in (db)",fontsize=10)
plt.xlabel(r"$\Omega$ (kHz)")
plt.subplot(1,2,2)
angles = np.angle(h)
plt.plot((fs * w/(2*pi))/1000.0,angles,'b')
plt.title(r"Angle",fontsize=12)
plt.ylabel(r"Angle (rad)",fontsize=10)
plt.xlabel(r"$\Omega$ (kHz)",fontsize=10)
plt.subplots_adjust(left=0.125, 
                    wspace=0.4)
plt.savefig("plots/Question1.png",bbox_inches="tight",pad=-1,format="png")

In [6]:
# impulse response of the filter
# forming the impulse input

pulse = np.zeros((200,1))
pulse[0] = 1

# initializing the impulse response
y = zeros_like(pulse)
time = np.linspace(0,len(pulse)*1.0/fs , 200, endpoint=False)

for n in range(len(pulse)):
    y[n] += b[0] * pulse[n]
    for k in range(1,len(a)):
        if (n-k)>=0:
            y[n] -= a[k] * y[n-k]


plt.figure()
plt.suptitle(r"Excitation Response",fontsize=12)
plt.subplot(1,2,1)
plt.plot(time,pulse,'b')
plt.title("Excitation Signal")
plt.ylabel(r"Amplitude",fontsize=10)
plt.xlabel(r"Time (sec)",fontsize=10)
plt.subplot(1,2,2)
plt.plot(time,y,'b')
plt.title("Impulse Response")
plt.ylabel(r"Amplitude",fontsize=10)
plt.xlabel(r"Time (sec)",fontsize=10)
plt.savefig("plots/Question1 Impulse Response.png",bbox_inches="tight",pad=-1,format="png")

#### Question 2
Excite the above resonator (“filter”) with a periodic source excitation of F0 = 140 Hz. You can approximate the source signal by narrow-triangular pulse train. Compute the output of the source-filter system over the duration of 0.5 second using the difference equation implementation of the LTI system. Plot the time domain waveform over a few pitch periods so that you can observe waveform characteristics. Play out the 0.5 sec duration sound and comment on the
sound quality. 

In [7]:
#formant_frequencies = [730, 1090, 2440]
#bandwidths= [100, 100, 100]
f1 = 900 #formant frequency
b1 = 200 #bandwidth
f_sampling = 16000
f_signal = 140
time = 0.5
#window_size = 20
#npoint_dft = 1024
t_sampling = 1/f_sampling
num_samples = int(f_sampling*time)

In [8]:
r = np.exp(-pi*b1*ts)
theta = 2*pi*f1*ts

poles = [r*exp(1j*theta) , r*exp(-1j*theta)]
zeros = 0

In [9]:
# coefficients in frequency domain
# b : numerator coefficients
# a : denominator coefficients
b,a = zpk2tf(zeros,poles,k=1)

In [10]:
t = np.linspace(0,time,num_samples)
#signal = sawtooth(f_signal*pi*t,width=0.5)

# sawtooth approximation using square
sig = square(2 * pi * f_signal* t, duty=0.01)+1

plt.figure()
plt.plot(t[:1000],sig[:1000])
plt.xlabel("$Time (sec)$",fontsize=10)
plt.ylabel("$Amplitude$",fontsize=10)
plt.title("Approximated Triangular Impulses")
plt.savefig("plots/Question2 Triangular Impulses.png",bbox_inches="tight",pad=-1,format="png")

In [11]:
y = zeros_like(sig)
# difference equation
for n in range(len(sig)):
    for k in range(len(b)):
            if (n-k)>=0:
                y[n] += b[k] * sig[n-k]
    for k in range(1,len(b)):
        if (n-k)>=0:
            y[n] += b[k] * sig[n-k]
    for k in range(1,len(a)):
        if (n-k)>=0:
            y[n] -= a[k] * y[n-k]


In [12]:
plt.figure()
plt.title("Excitation Response",fontsize=12)
plt.plot(t[:2514],y[:2514],'b')
plt.ylabel("Amplitude",fontsize=10)
plt.xlabel("Time (sec)",fontsize=10)
plt.savefig("plots/Question2 Response.png",bbox_inches="tight",pad=-1)

In [13]:
from scipy.io.wavfile import write
write("wavfiles/Q2output"+"_".join([str(f_signal),str(f1),str(b1)])+".wav",f_sampling,y)

#### Question 3
Vary the parameters as indicated below and comment on the differences in waveform and sound quality for the different parameter combinations.  

* (a) F0 = 120 Hz, F1 = 300 Hz, B1 = 100 Hz
* (b) F0 = 120 Hz, F1=1200 Hz, B1 = 200 Hz
* (c) F0 = 180 Hz, F1 = 300 Hz, B1 = 100 Hz

In [20]:
def generate_signal_response(time,sig,b,a):
    y = zeros_like(sig)
    # difference equation
    for n in range(len(sig)):
        for k in range(len(b)):
            if (n-k)>=0:
                y[n] += b[k] * sig[n-k]
        for k in range(1,len(a)):
            if (n-k)>=0:
                y[n] -= a[k] * y[n-k]
    return y

def plot_magnitude_response(b,a,f1,b1,f_signal):
    w,h = freqz(b,a)
    plt.figure()
    s = "Frequency response of vocal tract with F1: {}Hz and B1: {}Hz"
    plt.suptitle(s.format(f1,b1),fontsize=12)
    plt.subplot(1,2,1)
    plt.plot(fs * w/(2*pi),20*log10(abs(h)),'b')
    plt.title(r"Magnitude response",fontsize=12)
    plt.ylabel(r"$|H(\Omega|$",fontsize=10)
    plt.xlabel(r"$\Omega$")
    plt.subplot(1,2,2)
    angles = np.angle(h)
    plt.plot(fs * w/(2*pi),angles,'b')
    plt.title(r"Angle",fontsize=12)
    plt.ylabel(r"Angle (rad)",fontsize=10)
    plt.xlabel(r"$\Omega$",fontsize=10)
    plt.subplots_adjust(left=0.125,
                    wspace=0.4, )
    plt.savefig("plots/Q3_Freq_resp_"+str(f_signal)+"_"+str(f1)+"_"+str(b1)+".png",bbox_inches="tight",pad=-1,format="png")

def plot_and_save_waveform(t,y,f_signal,f1,b1,f_sampling):
    plt.figure()
    plt.title(r"Excitation Response",fontsize=12)
    plt.plot(t[:2514],y[:2514],'b')
    plt.ylabel(r"Amplitude",fontsize=10)
    plt.xlabel(r"Time (sec)",fontsize=10)
    plt.savefig("plots/Q3_Signal_Response"+str(f_signal)+str(f1)+"_"+str(b1)+".png",bbox_inches="tight",pad=-1,format="png")
    write("wavfiles/output"+"_".join([str(f_signal),str(f1),str(b1)])+".wav",f_sampling,y)

def generate_waveform(f1,b1,f_signal,fs=16000):
    time = 0.5 # total time duration
    ts = 1/fs # sampling time
    num_samples = int(f_sampling*time) # total number of signal samples
    r = np.exp(-pi*b1*ts) #radius in z-plane
    theta = 2*pi*f1*ts #angle in z-plane

    poles = [r*exp(1j*theta) , r*exp(-1j*theta)] #poles : 2 for every formant
    zeros = zeros_like(poles) # zeros 
    
    b,a = zpk2tf(zeros,poles,k=1)
    plot_magnitude_response(b,a,f1,b1,f_signal)
    t = np.linspace(0,time,num_samples)

    # sawtooth approximation using square
    sig = square(2 * pi * f_signal* t, duty=0.01)+1

    # 
    response = generate_signal_response(t,sig,b,a)
    plot_and_save_waveform(t,response,f_signal,f1,b1,fs)


In [21]:
formant_frequencies = [300, 1200, 300]
bandwidths= [100, 200, 100]
signal_frequencies = [120,120,180]

for i,j,k in list(zip(formant_frequencies,bandwidths,signal_frequencies)):
    generate_waveform(i,j,k)

<ipython-input-20-4db0b76cd569>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-20-4db0b76cd569>:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-20-4db0b76cd569>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-20-4db0b76cd569>:34: Runt

#### Question 4
In place of the simple single-resonance signal, synthesize the following more realistic vowel sounds at two distinct pitches (F0 = 120 Hz, F0 = 220 Hz). Keep the bandwidths constant at 100 Hz for all formants. Duration of sound: 0.5 sec  

**Vowel F1, F2, F3**
* /a/ 730, 1090, 2440
* /i/ 270, 2290, 3010
* /u/ 300, 870, 2240

(Optional: Use glottal pulse shaping and lip radiation filtering. Add a small amount of aspiration noise
and pitch jitter.)

In [17]:
def generate_signal_response(t,sig,b,a):
    y = zeros_like(sig)
    # difference equation
    for n in range(len(sig)):
        for k in range(len(b)):
            if (n-k)>=0:
                y[n] += b[k] * sig[n-k]
        for k in range(1,len(a)):
            if (n-k)>=0:
                y[n] -= a[k] * y[n-k]     
    return y

def plot_magnitude_response(b,a,vowel,f0):
    w,h = freqz(b,a)
    plt.figure()
    s = "Vocal tract response for vowel: /'{}'/ with signal freq: {}Hz"
    plt.suptitle(s.format(vowel,f0) ,fontsize=12,weight=2)
    plt.subplot(1,2,1)
    plt.plot(fs * w/(2*pi),20*log10(abs(h)),'b')
    plt.title("Magnitude response",fontsize=12)
    plt.ylabel(r"$|H(\Omega|$",fontsize=10)
    plt.xlabel(r"$\Omega$")
    plt.subplot(1,2,2)
    angles = np.angle(h)
    plt.plot(fs * w/(2*pi),angles,'b')
    plt.title(r"Angle",fontsize=12)
    plt.ylabel(r"Angle (rad)",fontsize=10)
    plt.xlabel(r"$\Omega$",fontsize=10)
    plt.subplots_adjust(left=0.125,
                    wspace=0.4)
    plt.savefig("plots/Q4_Freq_resp_"+vowel+"_"+str(f0)+".png",bbox_inches="tight",pad=-1,format="png")

def plot_and_save_waveform(t,y,f_signal,f_sampling,vowel):
    plt.figure()
    plt.title("Excitation",fontsize=12)
    plt.plot(t[:2514],y[:2514],'b')
    plt.ylabel("Impulse Response",fontsize=10)
    plt.xlabel("Time (sec)",fontsize=10)
    plt.savefig("plots/Q4_Signal_Response"+str(f_signal)+"_"+vowel+".png",bbox_inches="tight",pad=-1,format="png")
    write("wavfiles/output"+"_"+str(f_signal)+"_"+vowel+".wav",f_sampling,y)

def vocal_tract(formant_frequencies):
    global bw
    r = []
    theta = []
    for i in formant_frequencies:
        r.append(np.exp(-pi*bw*ts)) #radius in z-plane
        theta.append(2*pi*i*ts) #angle in z-plane

    denom_coeffs = []
    num_coeffs = []
    convolved_a = 1
    for radius,angle in zip(r,theta):
        poles = [radius*exp(1j*angle),radius*exp(-1j*angle)]
        zeros = zeros_like(poles)
        b,a = zpk2tf(zeros,poles,k=1)
        num_coeffs.append(b)
        denom_coeffs.append(a)
        convolved_a = conv(convolved_a,a)

    denom_coeffs = zeros_like(convolved_a)
    denom_coeffs[0] = 1
    
    return denom_coeffs,convolved_a

def generate_vowels(formant_frequencies,bandwidth,signal_frequency,vowel,time,f_sampling):
    ts = 1/f_sampling # sampling time
    num_samples = int(f_sampling*time) # total number of signal samples

    b,a = vocal_tract(formant_frequencies)
    plot_magnitude_response(b,a,vowel,signal_frequency)

    t = np.linspace(0,time,num_samples)

    # sawtooth approximation using square
    sig = square(2 * pi * signal_frequency* t, duty=0.01)+1

    response = generate_signal_response(t,sig,b,a)
    plot_and_save_waveform(t,response,signal_frequency,f_sampling,vowel)

In [18]:
f0 = [120,220]
f1 = [730,270,300]
f2 = [1090,2290,870]
f3 = [2440,3010,2240]
bw = 100
vow = ["a","i","u"]
duration = 0.5
fs = 16000 #sampling frequency
vowels = {}
for i in range(len(vow)):
    vowels[vow[i]] = {"formants":[f1[i],f2[i],f3[i]]}

for sig_freq in f0:
    for vowel in vowels:
        generate_vowels(vowels[vowel]["formants"],bw,sig_freq,vowel,duration,fs)

<ipython-input-17-a9f335e32b24>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
<ipython-input-17-a9f335e32b24>:34: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [19]:
### Google colab code to install tex
"""
! sudo apt-get install texlive-latex-recommended 
! sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended  
! wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip 
! unzip type1cm.zip -d /tmp/type1cm 
! cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins
! sudo mkdir /usr/share/texmf/tex/latex/type1cm 
! sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm 
! sudo texhash 
! apt install cm-super
! apt install dvipng
"""

'\n! sudo apt-get install texlive-latex-recommended \n! sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended  \n! wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip \n! unzip type1cm.zip -d /tmp/type1cm \n! cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins\n! sudo mkdir /usr/share/texmf/tex/latex/type1cm \n! sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm \n! sudo texhash \n! apt install cm-super\n! apt install dvipng\n'